# Задание

Нужно какое-то время пообучать трансформер для машинного перевода (те же данные что и на семинаре) и, используя, получившуюся модель перевести какой-то достаточно длинный текст на английском языке (например, какую-то новость). При этом нужно переделать функцию translate так, чтобы она обрабатывала не каждый текст по отдельности, а сразу целиком (или батчами если вы возьмете совсем длинный текст). На предложения текст можно поделить внутри или снаружи функции.

In [1]:
%%capture

!pip install tokenizers matplotlib sklearn

# Транформеры для решения seq2seq задач

Seq2seq - наверное самая общая формальная постановка задачи в NLP. Нужно из произвольной последовательности получить какую-то другую последовательность. И в отличие от разметки последовательности (sequence labelling) не требуется, чтобы обе последовательности совпадали по длине. Даже стандартную задачу классификации можно решать как seq2seq - можно рассматривать метку класса как последовательность длинны 1.

А трансформеры - sota архитектура для seq2seq задач. Мы не будем подробно разбирать устройство транформеров, если вам интересно вы можете поразбираться вот с этими материалами:

Оригинальная статья (сложновато) - https://arxiv.org/pdf/1706.03762.pdf

https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/  
https://jalammar.github.io/illustrated-transformer/

https://www.youtube.com/watch?v=iDulhoQ2pro

https://www.youtube.com/watch?v=TQQlZhbC5ps

Самый известный туториал (на торче) - https://nlp.seas.harvard.edu/2018/04/03/attention.html



Трансформеры будут подробно разбираться на курсе глубокого обучения (по выбору) на втором курсе.

Пока просто попробуем обучать модель на задаче машинного перевода. Для таких задач лучше всего использовать предобученные модели, но если у вас будет какая-то специфичная seq2seq задача, то имеет смысл попробовать обучить трансформер с нуля и в этой тертрадке вам нужно будет поменять только часть с загрузкой данных. 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
# text = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ')
# text = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ')
# f = open('opus.en-ru-train.ru.txt', 'w')
# f.write(text)
# f.close()

In [4]:
!gdown --id 1h4wmPd2A5J5gldpxjS3Oc4_JGYnEdWXn
!gdown --id 1w3NduhIXUwlN55exo616AGVVwQ9_55u6
!gdown --id 16hs2wBd6utkbWiLZA-AEnFwyzQE3LfW3

Downloading...
From: https://drive.google.com/uc?id=1h4wmPd2A5J5gldpxjS3Oc4_JGYnEdWXn
To: /content/model_tf.zip
256MB [00:01, 135MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w3NduhIXUwlN55exo616AGVVwQ9_55u6
To: /content/opus.en-ru-train.en.txt
67.8MB [00:01, 53.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=16hs2wBd6utkbWiLZA-AEnFwyzQE3LfW3
To: /content/opus.en-ru-train.ru.txt
121MB [00:01, 88.7MB/s]


Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [5]:
en_sents = open('opus.en-ru-train.en.txt').read().splitlines()
ru_sents = open('opus.en-ru-train.ru.txt').read().replace('\xa0', ' ').splitlines()

Пример перевода с английского на русский

In [6]:
en_sents[-1], ru_sents[-1]

('So what are you thinking?', 'Ну и что ты думаешь?')

Как обычно нам нужен токенизатор, а точнее даже 2, т.к. у нас два корпуса

In [7]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["opus.en-ru-train.en.txt"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru.txt"], trainer=trainer_ru)

### ВАЖНО!

Токенизатор - это неотъемлимая часть модели, поэтому не забывайте сохранять токенизатор вместе с моделью. Если вы забудете про это и переобучите токенизатор, то индексы токенов разойдутся и веса модели будут бесполезны. 

In [8]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [9]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

Переводим текст в индексы вот таким образом. В начало добавляем токен '[CLS]', а в конец '[SEP]'. Если вспомните занятие по языковому моделированию, то там мы добавляли "\<start>" и "\<end>" - cls и sep по сути тоже самое. Вы поймете почему именно cls и sep, а не start и end, если подробнее поразбираетесь с устройством трансформеров

In [10]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [11]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

3

In [12]:
# ограничимся длинной в 30 и 35 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_ru = 30, 35

In [13]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, max_len_ru) for t in ru_sents]

Паддинг внутри класса для датасета. Еще обратите внимание, что тут не стоит параметр batch_first=True как раньше

В торче принято, что размерность батча идет в конце и пример кода с трансформером расчитан на это. Конечно можно поменять сам код модели, но это сложнее, чем просто изменить тензор с данными.

In [14]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_ru):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)
        
        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, padding_value=PAD_IDX)

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_ru = self.texts_ru[:, index]

        return ids_en, ids_ru

Разбиваем на трейн и тест

In [15]:
X_en_train, X_en_valid, X_ru_train, X_ru_valid = train_test_split(X_en, X_ru, test_size=0.05)

In [16]:
training_set = Dataset(X_en_train, X_ru_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

In [17]:
valid_set = Dataset(X_en_valid, X_ru_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

# Код трансформера

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Там есть комментарии по каждому этапу

In [18]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

Обратите внимание на то как мы подаем данные в модель

In [19]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_ru) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) # чтобы батч был в конце
        texts_ru = texts_ru.T.to(DEVICE) # чтобы батч был в конце
        
        # помимо текста в модель еще нужно передать целевую последовательность
        # но не полную а без 1 последнего элемента
        # а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
        texts_ru_input = texts_ru[:-1, :]
        
        
        # в трансформерах нет циклов как в лстм 
        # каждый элемент связан с каждым через аттеншен
        # чтобы имитировать последовательную обработку
        # и чтобы не считать аттеншн с паддингом 
        # в трансформерах нужно считать много масок
        # подробнее про это по ссылкам выше
        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)
        logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()
        
        # сравниваем выход из модели с целевой последовательностью уже с этим последним элементом
        texts_ru_out = texts_ru[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []
    
    model.eval()  
    with torch.no_grad():
        for i, (texts_en, texts_ru) in enumerate(iterator):
            texts_en = texts_en.T.to(DEVICE)
            texts_ru = texts_ru.T.to(DEVICE)

            texts_ru_input = texts_ru[:-1, :]

            (texts_en_mask, texts_ru_mask, 
            texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)

            logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                           texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

            
            texts_ru_out = texts_ru[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
            epoch_loss.append(loss.item())
            
    return np.mean(epoch_loss)

In [20]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
RU_VOCAB_SIZE = tokenizer_ru.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, RU_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [21]:
torch.cuda.empty_cache()

In [22]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

Loss: 7.780176694869995;
Loss: 7.190528983592987;
Loss: 6.862414754231771;
Loss: 6.625997709989548;
Loss: 6.442283255386353;
Loss: 6.291036547819774;
Loss: 6.161857928139823;
Loss: 6.048613839626312;
Loss: 5.9464476958380805;
First epoch - 4.88803840637207, saving model..
Epoch: 1, Train loss: 5.899, Val loss: 4.888,            Epoch time=969.430s
Loss: 4.938443283081055;
Loss: 4.889075940132141;
Loss: 4.8441872873306275;
Loss: 4.8006130194664;
Loss: 4.757604760169983;
Loss: 4.715297318776448;
Loss: 4.6767846885408675;
Loss: 4.638347842335701;
Loss: 4.600490137788984;
Improved from 4.88803840637207 to 4.074344550132752, saving model..
Epoch: 2, Train loss: 4.583, Val loss: 4.074,            Epoch time=981.786s


KeyboardInterrupt: ignored

In [23]:
def translate(text):


    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_ru.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_ru_mask, 
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                   texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_ru.id_to_token(i).replace('##', '') for i in output_ids[1:]]))



In [24]:
translate("Example")

'Добавление'

In [25]:
translate('Can you translate that?')

'Вы можете , чтобы ты ?'

# Кастомная функция

In [27]:
%%capture 

!pip install spacy==3.0.5
!python3 -m spacy download en_core_web_sm
import spacy
model_en = spacy.load("en_core_web_sm")

In [36]:
import requests
url = 'https://raw.githubusercontent.com/vonCount/HSE-Computational-Linguistics-Course-2020-2021/main/sample_text.txt'
text = requests.get(url).text

In [37]:
print(text[:850])

Impact factor abandoned by Dutch university in hiring and promotion decisions
Faculty and staff members at Utrecht University will be evaluated by their commitment to open science.
A Dutch university says it is formally abandoning the impact factor — a standard measure of scientific success — in all hiring and promotion decisions. By early 2022, every department at Utrecht University in the Netherlands will judge its scholars by other standards, including their commitment to teamwork and their efforts to promote open science, says Paul Boselie, a governance researcher and the project leader for the university’s new Recognition and Rewards scheme. “Impact factors don’t really reflect the quality of an individual researcher or academic,” he says. “We have a strong belief that something has to change, and abandoning the impact factor is one 


In [46]:
def translate3(text):

    doc = model_en(text)
    in_list = list()
    out_list = list()

    for sent in doc.sents:
          sent = str(sent)
          input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(sent).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]    
          in_list.append(input_ids)
          out_list.append([tokenizer_ru.token_to_id('[CLS]')])

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(i) for i in in_list]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(i) for i in out_list]).to(DEVICE)
   
    (texts_en_mask, texts_ru_mask, 
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                   texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    pred_list =[]
    for i, logit in enumerate(torch.flatten(logits.argmax(2))):
      pred = logit.item()
      pred_list.append(pred)

    while any([pred[-1] not in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')] for pred in out_list]):
        for i in range(len(pred_list)):
          if out_list[i][-1] in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')]:              
              continue
          else:
              out_list[i].append(pred_list[i])

        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(i) for i in out_list]).to(DEVICE)

        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                      texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        pred_list =[]
        for i, logit in enumerate(torch.flatten(logits.argmax(2)[-1])):
          pred = logit.item()
          pred_list.append(pred)
        
    translations_list = []
    for i in out_list:
      translations_list.append(' '.join([tokenizer_ru.id_to_token(x).replace('##', '') for x in i[1:-1]]))

    return translations_list

In [47]:
translate3(text)

['В связи с помощью в рамках проекта « обеспечения безопасности и укрепления потенциала в качестве решения по - прежнему не менее , а также более , что в котором говорится о , что в настоящее',
 'В этом году и члены Совета по вопросам развития , которые будут представлены их приверженность вопросам , которые будут приняты для их мандата , а также на его пути к нему , в котором «',
 'В .',
 'В связи с помощью , говорит , что это предусмотрено , в результате чего бы не будет иметь возможность изменения климата , в которых будет проведено в все необходимые меры , что в настоящее время',
 'В ходе 20 22 22 , каждый из всех сотрудников , в котором будут приняты его учреждения , которые будут приняты другие учреждения , в том числе их усилия по вопросам , включая их приверженность',
 '« В этом не то , что не было бы очень важно , чтобы обеспечить , что он был очень мощ ным и на земле , что он говорит по - английски ): « В',
 '« Мы высоко оценивает , что - то , что - то есть , что - то , что ч